In [ ]:
import numpy as np
import cv2
import pandas as pd
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Add,Concatenate,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  GlobalAveragePooling2D,Dropout,Conv2D,MaxPool2D,UpSampling2D,LeakyReLU,AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
import keras.backend as K
from keras.layers import Layer
!pip install patchify
from patchify import patchify
from tensorflow.keras.layers import Layer, Dense, Embedding, MultiHeadAttention
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping

In [ ]:
train_load= np.load("DATA/X_train.npy")

In [ ]:
test_load =np.load("DATA/X_test.npy")

In [ ]:
Y_train=np.load("DATA/Y_train.npy")

In [ ]:
Y_test=np.load("DATA/Y_test.npy")

In [ ]:
print(train_load.shape)
print(test_load.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
train=[]
for i in tqdm(range(len(train_load))):
    image=train_load[i]
    patch_shape = (8, 8, 3)
    patches = patchify(image, patch_shape, 8)
    patches = np.reshape(patches, (784, 8, 8, 3))
    p_f=Flatten()(patches)
    train.append(p_f)


In [ ]:
test=[]
for j in tqdm(range(len(test_load))):
    image_t=test_load[j]
    patch_shape_t = (8, 8, 3)
    patches_t = patchify(image_t, patch_shape_t, 8)
    patches_t = np.reshape(patches_t, (784,8,8,3))
    p=Flatten()(patches_t)
    test.append(p)


In [ ]:
x1=np.array(train)
x2=np.array(test)

In [ ]:
Y=[]
for i in range(len(Y_train)):
    Y.append(int(Y_train[i]))


In [ ]:
FINAL_LABEL=[]
for j in range(len(Y)):
    if Y[j]==1:
        FINAL_LABEL.append([1,0,0,0])
    elif Y[j]==2:
        FINAL_LABEL.append([0,1,0,0])
    elif Y[j]==3:
        FINAL_LABEL.append([0,0,1,0])
    else:
        FINAL_LABEL.append([0,0,0,1])

In [ ]:
Y1=[]
for i in range(len(Y_test)):
    Y1.append(int(Y_test[i]))


In [ ]:
FINAL_LABEL_test=[]
for j in range(len(Y1)):
    if Y1[j]==1:
        FINAL_LABEL_test.append([1,0,0,0])
    elif Y1[j]==2:
        FINAL_LABEL_test.append([0,1,0,0])
    elif Y1[j]==3:
        FINAL_LABEL_test.append([0,0,1,0])
    else:
        FINAL_LABEL_test.append([0,0,0,1])

In [ ]:
y1=np.array(FINAL_LABEL)
y2=np.array(FINAL_LABEL_test)

In [ ]:
print(np.shape(train))
print(np.shape(test))
print(y1.shape)
print(y2.shape)

In [ ]:
class ClassToken(Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

In [ ]:
def mlp(x, cf):
    x = Dense(cf["mlp_dim"], activation="gelu")(x)
    x = Dropout(cf["dropout_rate"])(x)
    x = Dense(cf["hidden_dim"])(x)
    x = Dropout(cf["dropout_rate"])(x)
    return x

In [ ]:
def transformer_encoder(x, cf):
    skip_1 = x
    x = LayerNormalization()(x)
    x = MultiHeadAttention(
        num_heads=cf["num_heads"], key_dim=cf["hidden_dim"]
    )(x, x)
    x = Add()([x, skip_1])

    skip_2 = x
    x = LayerNormalization()(x)
    x = mlp(x, cf)
    x = Add()([x, skip_2])

    return x

In [ ]:
class selection_patches(Layer):
    def __init__(self,input_shapes,**kwargs):
        super(selection_patches,self).__init__(**kwargs)
        self.input_shapes = input_shapes
    def build(self,input_shape):
        self.W1=self.add_weight(name='attention_weight1', shape=(192,1),
                               initializer='random_normal', trainable=True)
        self.b1=self.add_weight(name='attention_bias1', shape=(1,),
                               initializer='zeros', trainable=True)
        self.W2=self.add_weight(name='attention_weight2', shape=(192,1),
                               initializer='random_normal', trainable=True)
        self.b2=self.add_weight(name='attention_bias2', shape=(1,),
                               initializer='zeros', trainable=True)
        self.W3=self.add_weight(name='attention_weight3', shape=(192,1),
                               initializer='random_normal', trainable=True)
        self.b3=self.add_weight(name='attention_bias3', shape=(1,),
                               initializer='zeros', trainable=True)

        self.W4=self.add_weight(name='attention_weight4', shape=(3,1),
                               initializer='random_normal', trainable=True)
        self.b4=self.add_weight(name='attention_bias4', shape=(1,),
                               initializer='zeros', trainable=True)
        super(selection_patches, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e1 = K.relu(K.dot(x,self.W1)+self.b1)
        e2 = K.relu(K.dot(x,self.W2)+self.b2)
        e3 = K.relu(K.dot(x,self.W3)+self.b3)
        e_head=tf.concat([e1, e2,e3],-1)
        e = K.relu(K.dot(e_head,self.W4)+self.b4)
        alpha = K.softmax(e)
        #print(alpha)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        #print("After:",alpha)
        # Compute the context vector
        #print(x)
        batch_size = tf.shape(x)[0]
        tensor1_shape = x.shape[1:]
        #print("tensor1 shape:",tensor1_shape)
        tensor2_tiled = tf.tile(alpha, [1, 1, tensor1_shape[0], tensor1_shape[1]])
        tensor1_reshaped = tf.reshape(x, [-1, 784, 1, 192])
        #print("tensor2:",tensor2_tiled )
        context = tensor1_reshaped * tensor2_tiled
        #print("CONTEXT:",context)
        context = K.sum(context, axis=1)

        sorted_indices = tf.argsort(context, direction='DESCENDING', axis=-1)
        top_64_indices = sorted_indices[:, :128]
        #print("output:",top_64_indices)
        return top_64_indices

In [ ]:

def ViT(cf):
    """ Inputs """
    input_shape = (784, cf["patch_size"]*cf["patch_size"]*cf["num_channels"])
    inputs = Input(input_shape)     ## (None, 256, 3072)
    inputss=selection_patches(input_shapes=784)(inputs)
    print(inputss)
    """ Patch + Position Embeddings """
    patch_embed = Dense(cf["hidden_dim"])(inputss)   ## (None, 256, 768)

    positions = tf.range(start=0, limit=cf["num_patches"], delta=1)
    pos_embed = Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions) ## (256, 768)
    embed = patch_embed + pos_embed ## (None, 256, 768)

    """ Adding Class Token """
    token = ClassToken()(embed)
    x = Concatenate(axis=1)([token, embed]) ## (None, 257, 768)

    for _ in range(cf["num_layers"]):
        x = transformer_encoder(x, cf)

    """ Classification Head """
    x = LayerNormalization()(x)     ## (None, 257, 768)
    x = x[:, 0, :]
    x = Dense(4, activation="softmax")(x)

    model = Model(inputs, x)
    return model



In [ ]:
hp={}
hp["num_layers"] = 12
hp["hidden_dim"] = 192
# hp["mlp_dim"] = 3072
hp["mlp_dim"] = 2048
# hp["num_heads"] = 12
hp["num_heads"] = 8
hp["dropout_rate"] = 0.1
hp["image_size"] = 224
hp["num_channels"] = 3
hp["patch_size"] = 8
# hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["num_patches"] = 128
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])
print(hp["flat_patches_shape"])

In [ ]:
model = ViT(hp)
model.summary()

In [ ]:
model.compile(
        loss="categorical_crossentropy",
        optimizer='adam',
        metrics=["acc"])

In [ ]:
callbacks = [
        ModelCheckpoint("high_acc.h5", monitor='val_loss', verbose=1, save_best_only=True),
        CSVLogger("model_details.csv"),
        EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),]

In [ ]:
r=model.fit(
        x1,y1,
        epochs=50,
        validation_data=(x2,y2),
        batch_size=8,
        callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt
# plot the l2wbb2wwnnnwwoss
plt.plot(r.history['loss'], label='Train loss')
plt.plot(r.history['val_loss'], label='Val loss')
plt.title('Model Loss')
plt.legend()
plt.show()

# plot the accuracy
plt.plot(r.history['acc'], label='Train acc')
plt.plot(r.history['val_acc'], label='Val acc')
plt.title('Model Accuracy')
plt.legend()
plt.show()

In [ ]:
model.save("/content/drive/MyDrive/DATA/high_acc.h5")

In [ ]:
base_model = model
# Choose a specific layer to extract features from
desired_layer = base_model.get_layer('layer_normalization_24')
feature_extractor_model = Model(inputs=base_model.input, outputs=desired_layer.output)

In [ ]:
F=[]
for i in tqdm(range(len(x2))):
    image=x2[i].reshape([-1,784,192])
    features=feature_extractor_model.predict(image)
    F.append(features)

In [ ]:
np.save("/content/drive/MyDrive/DATA/features_test_transformer.npy",F)